# Learning points:

- randomized selection algorithm
- randomized contraction algorithm

## Randomized selection algorithm

### Input:
- array: the list of length N in the unsorted order.
- i_idx: $i^{th}$ order-statistics that is of interest
    
    $0^{th}$-order statistics is the `min`

    $n^{th}$-order statistics is the `max`
- Output:
    the element of $i^{th}$ order statistics (with zero-based).
        
### High-level algorithm:
- base case if n is 1, return the element, otherwise
- choose pivot at uniformly random.
- partition around p, ( after partition, the left array contains any elements less than p, and on the right array contains any elements greater than p, such as the following: `[ < p ] [ p ] [ >p ]` )
- let j_idx be a new index of p (note that the result of each partition iteration is the correct position of the pivot, ie., the pivot is correctly positioned at its correct $i^{th}$ order statistics
- if j_idx == i-th order statistics (i_idx) that we're looking for, then bingo.. it's done
- if j_idx > i_idx, then we recurse again on the left side of the subarray, i,e `[ < p ]`
- if j_idx < i_idx, then we recurse again on the right side of the subarray, i.e `[ > p ]` with the $i^{th}$ order - $j^{th}$ order 


In [1]:
import pdb
import random


def rselect(array, i_idx, left=0, right=None):
    """Return ith-order statistics (zero-based) element from an array 
    using a randomized algorithm in selecting a pivot idx """
    
    if right is None:
        right = len(array) - 1
    
    if left >= right: 
        return array[0]
    
    p_idx = random.randint(left, right)
    j_idx = partition(array, left, right, p_idx)
    
    if j_idx == i_idx: 
        return array[i_idx]
    elif j_idx > i_idx:
        return rselect(array[:j_idx+1], i_idx)
    elif j_idx < i_idx:
        return rselect(array[j_idx:], i_idx - j_idx)
    
    
def partition(array, left, right, p_idx):
    array[left], array[p_idx] = array[p_idx], array[left]
    p_index = left + 1 
    p = array[left]
    should_swap = False
    
    for j in range(left+1, right+1):
        if array[j] > p:
            should_swap = True
        if array[j] <= p:
            if should_swap: 
                array[j], array[p_index] = array[p_index], array[j]
            p_index += 1
            
    array[p_index - 1], array[left] = array[left], array[p_index - 1]

    return p_index - 1
    
    
assert rselect([10, 8, 2, 4], 2) == 8
assert rselect([10, 8, 2, 4], 1) == 4
assert rselect([-10, 8, 2, 4, 7, 9, 10, 12, 13], 5) == 9
print "random selection seems to work"

random selection seems to work


## Randomized contraction algorithm

### Input:

- graph elements in a `txt` file which needs to be parsed into `adjacency list`

#### Graph representation by `adjacency list`
- adjacency list consists of:
    - a list of edges
    - a list of vertices

### High-level algorithm:

- While there are more than 2 vertices:
    - pick a remaining edge`(u,v)` uniformly at random
    - merge `u-v` vertices into a single vertex
    - remove self-loops if occurs
- return the min-cut as the number of edges within 2 vertices


In [2]:
import random
import sys


def parse_graph(filename):
    """parse graph from a file into adjacency list's graph representation
    
    This is assumed to be no parallel edges (a pair of edges that have a similar pair of vertices)
    
    Args: filename for a graph, it looks like as follows:
        1    2    4
        2    1    3    4
        
    Returns a tuple of vertices (list of vertices) and edges (list of edges)
    """
    vertices = []
    edges = set() # assume no parallel edges!
    with open(filename, "r") as f:
        for line in f:
            items = line.rstrip().split()
            v = int(items.pop(0))
            vertices.append(int(v))
            edges.update({tuple(sorted([v, int(i)])) for i in items})
    return vertices, list(edges)


def random_contraction(vertices, edges):
    while len(vertices) > 2:
        merged_edge = random.choice(edges)
        a, b = merged_edge 
        vertices.remove(a)
        new_edges = []
        for e in edges:
            if e == merged_edge: 
                # this will also remove the self-loop
                continue 
            if a in e:
                # reassign the edges end-points
                if e[0] == a:
                    other = e[1]
                if e[1] == a:
                    other = e[0]
                e = tuple(sorted([other, b])) 
            new_edges.append(e)
        edges = new_edges
    return vertices, edges 


def run_trial(n, vertices, edges):
    min_cut = sys.maxsize
    for _ in range(n):
        v, e = random_contraction(vertices[:], edges[:])
        if len(e) < min_cut:
            min_cut = len(e)
    return min_cut
                
    
assert parse_graph("./input/test_min_cuts.txt") == ([1, 2, 3, 4], [(1, 2), (2, 3), (3, 4), (2, 4), (1, 4)])
random_contraction([1, 2, 3, 4], [(1, 2), (1, 4), (2, 3), (2, 4), (3, 4)])
assert run_trial(10000, [1, 2, 3, 4], [(1, 2), (1, 4), (2, 3), (2, 4), (3, 4)]) == 2
print("contraction seems to work")

([2, 4], [(2, 4), (2, 4), (2, 4)])

contraction seems to work
